## Problem 04
Find the solution to: <br>
\begin{equation}
\begin{aligned}
& \underset{X}{\text{min}}
& & f = x_1^2 + x_2^2 + x^2_3 \\
& \text{subject to}
& & h_1 = \frac{x_1^2}{4} + \frac{x_2^2}{5} + \frac{x_3^2}{25} - 1 = 0 \\
& & & h_2 = x_1 + x_2 - x_3 = 0
\end{aligned}
\end{equation}

With two contraints and three variables, there will be two state variables and one driving variable. I will make $x_1\ $ my driving variable and $x_2\ $ and $x_3\ $ my state variables<br>

The following equations define the reduced gradient used to solve the problem
\begin{equation}
\begin{aligned}
d &= x_1 \\
s &= \begin{bmatrix} x_2 & x_3 \end{bmatrix} \\
\frac{df}{dd} &= \frac{\partial f}{\partial d} - \frac{\partial f}{\partial s} \left(\frac{\partial h}{\partial s}\right)^{-1} \frac{\partial h}{\partial d} \\
\frac{\partial f}{\partial d} &= 2x_1 \\
\frac{\partial f}{\partial s} &= \begin{bmatrix} 2x_2 & 2x_3 \end{bmatrix} \\
\frac{\partial h}{\partial d} &= \begin{bmatrix} \frac{1}{2} x_2 \\ 1 \end{bmatrix} \\
\frac{\partial h}{\partial s} &= \begin{bmatrix} \frac{2}{5} x_2 & \frac{2}{25} x_3 \\ 1 & -1 \end{bmatrix}
\end{aligned}
\end{equation}

A line search was implemented to determine the step size to take and the newton-ralphson method was used to ensure that the constraints were followed.

In [6]:
import numpy as np

x1_0 = 1  # initial driving condition
x2_0 = 1.561370113  # initial state conditions
x3_0 = 2.561370113
X = np.array([x2_0, x3_0, x1_0])
k = 0  # initialize counter for solver
eps = 10 ** -3  # minimum error for slope
sk = np.array([X[0], X[1]])  # state variables
dk = np.array([X[2]])  # driving variable
dfdx1 = np.array([2 * X[2]])  # dfdd (partial)
dfds = np.array([2 * X[0], 2 * X[1]])  # dfds (partial)
dhds = np.array([[2 / 5 * X[0], 2 / 25 * X[1]],
                 [1, -1]])
dhdd = np.array([[1 / 2 * X[2]], [1]])
dfdd = dfdx1 - np.matmul(np.matmul(dfds, np.linalg.inv(dhds)), dhdd)  # non-linear dfdd


def minfunc(X):  # calculation of function to be minimized
    x1 = X[2]
    x2 = X[0]
    x3 = X[1]
    return x1 ** 2 + x2 ** 2 + x3 ** 2


def confunc(X):  # claclulation of constraint conditions
    x1 = X[2]
    x2 = X[0]
    x3 = X[1]
    h1 = x1 ** 2 / 4 + x2 ** 2 / 5 + x3 ** 2 / 25 - 1
    h2 = x1 + x2 - x3
    return np.array([[h1], [h2]])


def linesearch(dfdd, sk, dk, dhds_line, dhdd_line):  # inexact line search
    alpha = 1
    if dfdd < 0:
        alpha = -0.1
    b = 0.4
    t = 0.2
    counter = 0
    dhds_inv = np.linalg.inv(dhds_line)  # calculates inverse of dhds for function evaluation
    skstep_matrix = np.transpose(np.matmul(np.matmul(dhds_inv, dhdd_line), dfdd))
    sk1 = sk + alpha * skstep_matrix  # state step
    dk1 = dk - alpha * dfdd  # driving step
    X_step = np.concatenate((sk1, dk1), axis=None)
    f = minfunc(X_step)  # function analysis
    phi = minfunc(np.concatenate((sk, dk), axis=None)) - alpha * t * (dfdd)**2  # phi analysis with step
    while f > phi and counter < 50:
        alpha = b * alpha
        sk1 = sk + alpha * skstep_matrix
        dk1 = dk - alpha * dfdd
        X_step = np.concatenate((sk1, dk1), axis=None)
        f = minfunc(X_step)
        phi = minfunc(np.concatenate((sk, dk), axis=None)) - alpha * t * (dfdd)**2
        counter += 1
    if counter == 50:
        print("linesearch failed to converge")
    return alpha, skstep_matrix


def solvefunc(driving, state_approx):  # Newton rhobson method for finding new state values
    c = 0
    x1 = driving
    x2 = state_approx[0]
    x3 = state_approx[1]
    skactual = np.array([x2, x3])
    Xpart = np.concatenate((state_approx, driving), axis=None)
    h = confunc(Xpart)
    while np.linalg.norm(h) > 10 ** -3 and c < 30:
        sktran = np.reshape(skactual, (2,1)) - np.matmul(
            np.linalg.inv(np.array([[2 / 5 * Xpart[0], 2 / 25 * Xpart[1]], [1, -1]])), confunc(Xpart))
        x2 = sktran[0]
        x3 = sktran[1]
        skactual = np.concatenate((x2, x3), axis=None)
        Xpart = np.concatenate((x2, x3, x1), axis=None)
        h = confunc(Xpart)
        c += 1
    if c == 30:
        print("Newton Method did not Converge")
    return skactual


while np.linalg.norm(dfdd) > eps and k < 30:  # solver loop
    alphak, stepmatrix = linesearch(dfdd, sk, dk, dhds, dhdd)
    dk = dk - alphak * dfdd
    sk_lin_approx = sk + alphak * stepmatrix
    sk = solvefunc(dk, sk_lin_approx)
    X = np.concatenate((sk, dk), axis=None)
    dfdx1 = np.array([2 * X[2]])
    dfds = np.array([2 * X[0], 2 * X[1]])
    dhds = np.array([[2 / 5 * X[0], 2 / 25 * X[1]],
                     [1, -1]])
    dhdd = np.array([[1 / 2 * X[2]], [1]])
    dfdd = dfdx1 - np.matmul(np.matmul(dfds, np.linalg.inv(dhds)), dhdd)
    k += 1
    print(X, dfdd)
sol = minfunc(X)
print("x1 = " + np.array2string(X[2]) + "\nx2 = " + np.array2string(X[0]) + "\nx3 = " + np.array2string(
    X[1]) + "\ndfdd = " + np.array2string(dfdd) + "\nf(x) = " + np.array2string(sol) )

[1.60058809 2.55444082 0.95385274] [0.31335073]
[1.6939502  2.52246265 0.82851245] [0.79428032]
[1.87955651 2.39035683 0.51080032] [1.75785527]
[ 2.0625786   1.87023681 -0.19234179] [2.9244532]
[ 1.63422785  0.27210478 -1.36212307] [1.5413768]
[ 1.5270714   0.06630022 -1.46077118] [0.96130418]
[ 1.44991486 -0.07237979 -1.52229465] [0.49065306]
[ 1.40797086 -0.14572559 -1.55369645] [0.21034409]
[ 1.38872461 -0.17843386 -1.56715847] [0.0772124]
[ 1.37957148 -0.19252858 -1.57210006] [0.01776596]
[ 1.37787271 -0.19536437 -1.57323708] [0.00576354]
[ 1.37732012 -0.19628583 -1.57360595] [0.00185442]
[ 1.37714217 -0.19658246 -1.57372463] [0.00059505]
x1 = -1.57372463
x2 = 1.37714217
x3 = -0.19658246
dfdd = [0.00059505]
f(x) = 4.41177444


After running the simulation the final results ended with a function evaluation of 4.412. This occures at x1 = -1.574, x2 = 1.377, and x3 = -0.197 <br>
